In [1]:
# Required to setup DB and consume GTFS static data
import mysql.connector
from mysql.connector import Error
from mysql.connector import errorcode
from mysql.connector.constants import ClientFlag
import os
import csv

STATIC_CSV = "static_.csv"
DB_CSV = "db_.csv"

In [2]:
def executeScriptsFromFile(filename, cursor):
    # Open and read the file as a single buffer
    fd = open(filename, 'r')
    sqlFile = fd.read()
    fd.close()

    # all SQL commands (split on ';')
    sqlCommands = sqlFile.split(';')

    # Execute every command from the input file
    for command in sqlCommands:
        # This will skip and report errors
        # For example, if the tables do not yet exist, this will skip over
        # the DROP TABLE commands
        try:
            cursor.execute(command)
        except mysql.connector.Error as error:
            print("Error occurred while executing script : ", error)

In [3]:
## Create Tables and Indexes

def db_setup(filepath, tab=True, idx=False):
    connection = mysql.connector.connect(host='localhost', user='root', password='admin')
    try:
        cursor = connection.cursor()
        print("FILEPATH received : ", filepath)
        createtab_file = filepath + "create-tables.sql"
        createidx_file = filepath + "create-index.sql"
        
        if(tab):
            executeScriptsFromFile(createtab_file, cursor)
            print("TABLES CREATED ...")
            connection.commit()
        
        if(idx):
            executeScriptsFromFile(createidx_file, cursor)
            print("INDEXES CREATED ...")
            connection.commit()
    
    except mysql.connector.Error as error:
        connection.rollback() #rollback if any exception occured
        print("The following error has occurred ... ".format(error))

    finally:
        #closing database connection.
        if(connection.is_connected()):
            cursor.close()
            connection.close()
            print("DB Setup complete. MySQL connection is closed.")

In [4]:
def all_tables(path, folder, date):
    filepath = "\'" + path + folder + "/"
    
    # Agency
    filename = "agency.txt'"
    tablename = "gtfs.agency"
    print(filepath)
    print(filename)
    print(tablename)
    bulk_ins(filepath, filename, tablename)
    
    # Calendar_Dates
    filename = "calendar_dates.txt'"
    tablename = "gtfs.calendar_dates"
    bulk_ins(filepath, filename, tablename)
    
    # Routes
    filename = "routes.txt'"
    tablename = "gtfs.routes"
    bulk_ins(filepath, filename, tablename)
    
    # Shapes
    filename = "shapes.txt'"
    tablename = "gtfs.shapes"
    bulk_ins(filepath, filename, tablename)
    
    # Stop_Times
    filename = "stop_times.txt'"
    tablename = "gtfs.stop_times"
    bulk_ins(filepath, filename, tablename, date)
    
    #Stops
    filename = "stops.txt'"
    tablename = "gtfs.stops"
    bulk_ins(filepath, filename, tablename)
    
    #Transfers
    filename = "transfers.txt'"
    tablename = "gtfs.transfers"
    bulk_ins(filepath, filename, tablename)
    
    #Trips
    filename = "trips.txt'"
    tablename = "gtfs.trips"
    bulk_ins(filepath, filename, tablename, date)

In [5]:
def bulk_ins(filepath, filename, tablename, date=None):
    try:
        connection = mysql.connector.connect(host='localhost', database='gtfs', user='root', \
                                             password='admin', client_flags=[ClientFlag.LOCAL_FILES])
        print("Connected to DB ...", ClientFlag.LOCAL_FILES)
        
        # Create cursor and execute Load SQL
        autoc_sql = "SET autocommit=0;"
        ucheck_sql = "SET unique_checks=0;"
        fcheck_sql = "set foreign_key_checks=0;"
        logcheck_sql = "set sql_log_bin=0;"
        
        load_sql = ("LOAD DATA LOCAL INFILE " + filepath + filename +
                    " INTO TABLE " + tablename +
                    " FIELDS TERMINATED BY ','"
                    " OPTIONALLY ENCLOSED BY '\"'"
                    " LINES TERMINATED BY '\\n'"
                    " IGNORE 1 LINES")
        
        # INSERT DATE IN TABLES STOP_TIMES AND TRIPS
        if(tablename == "gtfs.stop_times" or tablename == "gtfs.trips"):
            print("DATE VALUE RECEIVED ... ", date)
            load_sql = load_sql + " SET trip_date = '" + str(date) + "';"
        else:
            load_sql = load_sql + ";"
        
        cursor = connection.cursor()
        print("BULK INS ... ")
        print("FILEPATH received : ", filepath.strip('\''))
        print(filepath + filename)
        print(load_sql)
        
        cursor.execute(autoc_sql)
        cursor.execute(ucheck_sql)
        cursor.execute(fcheck_sql)
        cursor.execute(logcheck_sql)
        
        cursor.execute(load_sql)
        connection.commit()
        print("Succuessfully loaded the table " + tablename + " from " + filename.strip('\'') + " ... ")
    
    except mysql.connector.Error as error :
        print(cursor.statement)
        connection.rollback() #rollback if any exception occured
        print("Failed inserting record into table " + tablename + " from " + filename.strip('\'') + " ... {}".format(error))
        
    finally:
        #closing database connection.
        if(connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [22]:
def main():

    # FETCH FILES
    cwd = os.getcwd()
    db_csv = cwd.replace("\\", "/") + "/" + DB_CSV
    static_csv = cwd.replace("\\", "/") + "/" + STATIC_CSV
    
#     # GET DB SCRIPTS PATH
#     db_scripts = ""
#     with open(db_csv, "r") as file:
#         next(file)
#         for row in file:
#             db_scripts = row.strip().replace("\\", "/") + "/"
    
#     # SETUP DB
#     db_setup(db_scripts, tab=True, idx=False)
    
#     # DUMP DATA TO DB
#     dataset_path = ""
#     folder_name = ""
#     folder_date = ""
#     with open(static_csv, "r") as file:
#         next(file)
#         for row in file:
#             row = row.strip().split(",")
#             dataset_path = row[0].replace("\\", "/") + "/"
#             folder_name = row[1]
#             folder_date = row[2]
#             all_tables(dataset_path, folder_name, folder_date)
    
if __name__ == "__main__":
    main()

FILEPATH received :  C:/Users/kakka/Documents/GTFS_/code/
TABLES CREATED ...
DB Setup complete. MySQL connection is closed.
'C:/Users/kakka/Documents/GTFS_/static_data/NL-20190109.gtfs/
agency.txt'
gtfs.agency
Connected to DB ... 128
BULK INS ... 
FILEPATH received :  C:/Users/kakka/Documents/GTFS_/static_data/NL-20190109.gtfs/
'C:/Users/kakka/Documents/GTFS_/static_data/NL-20190109.gtfs/agency.txt'
LOAD DATA LOCAL INFILE 'C:/Users/kakka/Documents/GTFS_/static_data/NL-20190109.gtfs/agency.txt' INTO TABLE gtfs.agency FIELDS TERMINATED BY ',' OPTIONALLY ENCLOSED BY '"' LINES TERMINATED BY '\n' IGNORE 1 LINES;
Succuessfully loaded the table gtfs.agency from agency.txt ... 
MySQL connection is closed
Connected to DB ... 128
BULK INS ... 
FILEPATH received :  C:/Users/kakka/Documents/GTFS_/static_data/NL-20190109.gtfs/
'C:/Users/kakka/Documents/GTFS_/static_data/NL-20190109.gtfs/calendar_dates.txt'
LOAD DATA LOCAL INFILE 'C:/Users/kakka/Documents/GTFS_/static_data/NL-20190109.gtfs/calendar_

Succuessfully loaded the table gtfs.trips from trips.txt ... 
MySQL connection is closed


In [ ]:
# Segregate Index Creation for the DB as it takes up a lot of time
# Run after all data has been inserted

# # FETCH FILES
# cwd = os.getcwd()
# db_csv = cwd.replace("\\", "/") + "/" + DB_CSV
# static_csv = cwd.replace("\\", "/") + "/" + STATIC_CSV

# # GET DB SCRIPTS PATH
# db_scripts = ""
# with open(db_csv, "r") as file:
#     next(file)
#     for row in file:
#         db_scripts = row.strip().replace("\\", "/") + "/"
# db_setup(db_scripts, tab=False, idx=True)

In [7]:
# Generate a set of usable trip and service ids from the DB
def push_routes(filepath):
    try:
        connection = mysql.connector.connect(host='localhost', database='gtfs', user='root', \
                                             password='admin', client_flags=[ClientFlag.LOCAL_FILES])
        print("Connected to DB ...", ClientFlag.LOCAL_FILES)
        
        #
        sel_sql = ( " SELECT DISTINCT ROUTE_SHORT_NAME, "
                " CASE ROUTE_TYPE "
                " WHEN '0' THEN 'TRAM'"
                " WHEN '1' THEN 'SUBWAY'"
                " WHEN '2' THEN 'RAIL'"
                " WHEN '3' THEN 'BUS'"
                " WHEN '4' THEN 'FERRY'"
                " END AS ROUTE_TYPE"
                " FROM GTFS.ROUTES ORDER BY ABS(ROUTE_SHORT_NAME), ROUTE_TYPE;" )
        
        cursor = connection.cursor()
        print("push_routes() ... ")
        print("FILEPATH received : ", filepath)
        print(sel_sql)
        cursor.execute(sel_sql)
        all_rows = cursor.fetchall()
        
        with open(filepath + 'routes_list.csv', 'w') as myfile:
            for row in all_rows:
                myfile.write(row[0] + "," + row[1] + "\n")
            print("Data write success ... ")
            print("Please check CSV file ROUTES.CSV at " + filepath)
            
#         with open(filepath + 'routes_list.csv', 'w') as myfile:
#             wr = csv.writer(myfile)
#             for row in all_rows:
#                 wr.writerow(row)
#             print("Data write success ... ")
#             print("Please check CSV file ROUTES.CSV at " + filepath)
    
    except mysql.connector.Error as error :
        print(cursor.statement)
        connection.rollback() #rollback if any exception occured
        print("Failed fetching data from GTFS.ROUTES ... {}".format(error))
        
    finally:
        #closing database connection.
        if(connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

FILEPATH received :  C:/Users/kakka/Documents/GTFS_/code/
INDEXES CREATED ...
DB Setup complete. MySQL connection is closed.


In [8]:
# GENERATE ROUTE DETAILS FOR USER
cwd = os.getcwd()
print(cwd)
push_routes(cwd.replace("\\", "/") + "/")

C:\Users\kakka\Documents\GTFS_\code
Connected to DB ... 128
push_routes() ... 
FILEPATH received :  C:/Users/kakka/Documents/GTFS_/code/
 SELECT DISTINCT ROUTE_SHORT_NAME,  CASE ROUTE_TYPE  WHEN '0' THEN 'TRAM' WHEN '1' THEN 'SUBWAY' WHEN '2' THEN 'RAIL' WHEN '3' THEN 'BUS' WHEN '4' THEN 'FERRY' END AS ROUTE_TYPE FROM GTFS.ROUTES ORDER BY ABS(ROUTE_SHORT_NAME), ROUTE_TYPE;
Data write success ... 
Please check CSV file ROUTES.CSV at C:/Users/kakka/Documents/GTFS_/code/
MySQL connection is closed


In [9]:
# Required to parse GTFS real-time feed
from google.transit import gtfs_realtime_pb2
import requests

In [53]:
def read_userpref(filepath):
    try:
        connection = mysql.connector.connect(host='localhost', database='gtfs', user='root', \
                                             password='admin', client_flags=[ClientFlag.LOCAL_FILES])
        print("Connected to DB ...", ClientFlag.LOCAL_FILES)
        
        # Open and read the file as a single buffer
        fd = open(filepath + 'user_routes.csv', 'r')
        usrFile = fd.read()
        fd.close()
        
        lines = usrFile.split("\n")
        print(lines)
        
        routes = []

        sel_sql = ( " SELECT DISTINCT ROUTE_ID"
                    " FROM GTFS.ROUTES"
                    " WHERE ROUTE_TYPE = %s"
                    " AND ROUTE_SHORT_NAME = %s"
                    " ORDER BY ROUTE_ID;")
        
        cursor = connection.cursor()
        print("read_userpref() ... ")
        print("FILEPATH received : ", filepath)
        print(sel_sql)
        
        for line in lines:
            line = line.split(',')
            print(line)
            # REVERSE MAP ROUTE TYPE
#             " WHEN '0' THEN 'TRAM'"
#             " WHEN '1' THEN 'SUBWAY'"
#             " WHEN '2' THEN 'RAIL'"
#             " WHEN '3' THEN 'BUS'"
#             " WHEN '4' THEN 'FERRY'"
            if line[1] == 'TRAM':
                line[1] = 0
            elif line[1] == 'SUBWAY':
                line[1] = 1
            elif line[1] == 'RAIL':
                line[1] = 2
            elif line[1] == 'BUS':
                line[1] = 3
            elif line[1] == 'FERRY':
                line[1] = 4
            cursor.execute(sel_sql, (line[1], line[0]))
            routes.extend(a[0] for a in cursor.fetchall())
            
        return routes
    
    except mysql.connector.Error as error :
        print(cursor.statement)
        connection.rollback() #rollback if any exception occured
        print("Failed inserting record into table " + tablename + " from " + filename + " ... {}".format(error))
        
    finally:
        #closing database connection.
        if(connection.is_connected()):
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

In [65]:
# Read User Input for lines from a CSV

cwd = os.getcwd()
print(cwd)
user_routes = read_userpref(cwd.replace("\\", "/") + "/")
print(user_routes)

feed = gtfs_realtime_pb2.FeedMessage()
response = requests.get('http://gtfs.ovapi.nl/nl/tripUpdates.pb')

if(response.status_code == 200):
    feed.ParseFromString(response.content)
    for entity in feed.entity:
      if entity.HasField('trip_update'):
        for ur in user_routes:
#             ur = '\'' + str(ur) + '\''
#             print(ur)
            if(entity.trip_update.trip.route_id == str(ur)):
                print("entity.id --- ", entity.id)
                print(entity.trip_update)
                print(entity.ID_FIELD_NUMBER, entity.IS_DELETED_FIELD_NUMBER, \
                      entity.TRIP_UPDATE_FIELD_NUMBER, entity.VEHICLE_FIELD_NUMBER, \
                      entity.ALERT_FIELD_NUMBER, entity.DESCRIPTOR)
else:
    print("ERROR FETCHING REALTIME DATA FROM THE gtfs.ovapi.nl server")
    print(response.reason)

C:\Users\kakka\Documents\GTFS_\code
Connected to DB ... 128
['1,BUS', '1,FERRY', '1,TRAM', '19,BUS', '19,FERRY', '19,TRAM']
read_userpref() ... 
FILEPATH received :  C:/Users/kakka/Documents/GTFS_/code/
 SELECT DISTINCT ROUTE_ID FROM GTFS.ROUTES WHERE ROUTE_TYPE = %s AND ROUTE_SHORT_NAME = %s ORDER BY ROUTE_ID;
['1', 'BUS']
['1', 'FERRY']
['1', 'TRAM']
['19', 'BUS']
['19', 'FERRY']
['19', 'TRAM']
MySQL connection is closed
[188, 211, 1063, 1744, 2644, 6731, 7170, 19413, 32759, 36962, 41487, 45366, 45413, 45415, 52890, 52901, 52914, 54174, 54278, 54549, 54567, 57599, 57606, 57610, 57656, 57689, 57757, 58941, 59833, 60805, 61331, 61767, 61784, 61933, 61942, 61944, 62272, 57850, 59297, 60662, 7213, 42296, 57580, 57792, 62392, 18931, 60664, 62498]
entity.id ---  2019-01-17:CXX:Z001:58
trip {
  trip_id: "84524026"
  start_time: "22:01:00"
  start_date: "20190117"
  schedule_relationship: SCHEDULED
  route_id: "7170"
  direction_id: 1
}
stop_time_update {
  stop_sequence: 0
  departure {
   

1 2 3 4 5 <google.protobuf.descriptor.Descriptor object at 0x000001E722F10588>
entity.id ---  2019-01-17:SYNTUS:3801:36915
trip {
  trip_id: "84457648"
  start_time: "22:25:00"
  start_date: "20190117"
  schedule_relationship: SCHEDULED
  route_id: "57689"
  direction_id: 0
}
stop_time_update {
  stop_sequence: 1
  departure {
    delay: 53
    time: 1547760353
  }
  stop_id: "1237693"
}
stop_time_update {
  stop_sequence: 2
  arrival {
    delay: 55
    time: 1547760429
  }
  departure {
    delay: 55
    time: 1547760429
  }
  stop_id: "868302"
}
stop_time_update {
  stop_sequence: 3
  arrival {
    delay: 38
    time: 1547760518
  }
  departure {
    delay: 38
    time: 1547760518
  }
  stop_id: "867111"
}
stop_time_update {
  stop_sequence: 5
  arrival {
    delay: 0
    time: 1547760599
  }
  departure {
    delay: 0
    time: 1547760599
  }
  stop_id: "868298"
}
stop_time_update {
  stop_sequence: 11
  arrival {
    delay: -65
    time: 1547761015
  }
  departure {
    delay: -65